In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
data=pd.read_csv("mnist_test.csv")
data.head()

,7,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
data=np.array(data)
m,n=data.shape
np.random.shuffle(data)

data_dev=data[0:1000].T #each column is a training example now.
X_dev=data_dev[1:n]
y_dev=data_dev[0]

data_train=data[1000:m].T
X_train=data_train[1:n]
y_train=data_train[0]

X_train = X_train / 255.0
X_dev = X_dev / 255.0

In [12]:
def init_params():
    W1 = np.random.randn(10, 784) * np.sqrt(1/784)
    b1 = np.zeros((10,1))
    W2 = np.random.randn(10, 10) * np.sqrt(1/10)
    b2 = np.zeros((10,1))
    return W1, b1, W2, b2

def ReLU(Z):
  return np.maximum(Z,0)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return expZ / np.sum(expZ, axis=0, keepdims=True)

def forward_prop(W1,b1,W2,b2,X):
  Z1=W1.dot(X)+b1
  A1=ReLU(Z1)
  Z2=W2.dot(A1)+b2
  A2=softmax(Z2)
  return Z1,A1,Z2,A2

def one_hot(y):
  one_hot_y=np.zeros((y.size,y.max()+1))
  one_hot_y[np.arange(y.size),y]=1
  one_hot_y=one_hot_y.T
  return one_hot_y

def ReLU_deriv(Z):
  return Z>0

def back_prop(Z1, A1, Z2, A2, W1, W2, X, y):
    one_hot_y = one_hot(y)
    m = y.size
    dZ2 = A2 - one_hot_y
    dW2 = (1 / m) * dZ2.dot(A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = (1 / m) * dZ1.dot(X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [13]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    correct = np.sum(predictions == Y)
    accuracy = correct / Y.size * 100  # float, always between 0 and 100
    print(f"Predictions: {predictions}, Labels: {Y}")
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy


def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 500 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [14]:
W1, b1, W2, b2 = gradient_descent(X_train, y_train, 0.1, 1000)

Iteration:  0
Predictions: [0 0 0 ... 0 0 0], Labels: [8 0 2 ... 8 9 6]
Accuracy: 10.48%
10.478942104678298
Iteration:  500
Predictions: [8 0 2 ... 7 9 6], Labels: [8 0 2 ... 8 9 6]
Accuracy: 91.17%
91.16568507611957


In [15]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

In [16]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, y_dev)

Predictions: [7 1 8 4 3 1 2 1 6 9 5 7 6 2 4 3 7 3 9 2 6 1 1 0 2 3 3 2 2 1 0 8 8 6 7 4 5
 3 5 2 4 7 7 3 6 1 8 0 9 5 4 6 0 7 7 4 6 6 0 6 6 6 1 0 8 0 1 8 9 3 0 1 8 9
 2 0 3 0 8 3 5 8 1 1 4 1 6 2 6 5 5 6 4 5 2 7 8 8 1 0 5 4 3 9 9 3 5 8 6 8 4
 6 6 2 3 5 7 1 4 5 4 5 2 8 3 8 5 8 4 2 0 6 8 9 9 5 5 8 7 6 7 8 8 8 1 6 6 3
 7 1 7 0 5 1 7 7 5 8 5 6 6 6 8 7 9 4 7 8 1 8 7 6 0 1 1 6 8 2 4 1 0 7 0 8 7
 7 1 6 0 0 5 0 3 9 4 4 8 0 5 0 7 3 8 9 6 5 3 9 4 7 8 5 6 1 4 8 8 3 8 5 9 3
 6 1 7 9 8 5 5 2 9 7 3 3 7 4 2 8 9 5 5 4 4 1 7 0 1 3 9 6 5 4 6 5 2 5 4 0 4
 6 0 2 9 9 4 9 0 4 4 2 4 4 7 4 4 7 3 6 1 0 4 6 9 5 7 4 4 2 5 8 0 7 7 0 7 1
 1 6 8 1 7 2 7 6 9 0 7 1 0 3 9 9 8 2 1 9 5 7 9 5 0 9 2 1 7 3 6 2 9 8 8 4 8
 3 8 7 6 6 4 4 8 5 0 3 5 4 8 3 9 6 0 9 0 4 6 8 8 5 7 2 2 1 8 9 7 1 6 9 6 9
 2 1 3 3 3 6 1 1 4 9 2 5 9 9 1 1 2 8 5 7 7 0 7 6 5 0 5 8 1 5 0 1 3 2 5 0 4
 5 9 4 5 0 0 8 2 7 9 4 7 4 5 3 2 0 7 8 1 3 3 3 4 7 4 0 9 8 5 3 3 3 2 4 1 5
 5 0 6 1 9 7 4 5 2 1 8 9 2 2 4 1 1 4 5 5 8 9 6 1 1 9 8 1 6 8 3 1 4 7 2 4 9
 0 2 1 0 5 1

90.5